# SC1015 Mini-Project: What pre-exisiting conditions increases a patient's vulnerability to mortality from Covid-19?

### Essential Libraries

> NumPy : Library for Numeric Computations in Python  
> Pandas : Library for Data Acquisition and Preparation  
> Matplotlib : Low-level library for Data Visualization  
> Seaborn : Higher-level library for Data Visualization  
> Sklearn : Machine Learning and Statistical Modeling

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
sb.set() # set the default Seaborn style for graphics
pd. set_option('display.max_columns', None)

## Exploratory Data Analysis

In [2]:
#Import Dataset
covid_data = pd.read_csv('Dataset/Covid Data.csv')
covid_data.head(100)

USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0       2             1    1             1  03/05/2020       97          1   
1       2             1    2             1  03/06/2020       97          1   
2       2             1    2             2  09/06/2020        1          2   
3       2             1    1             1  12/06/2020       97          2   
4       2             1    2             1  21/06/2020       97          2   
..    ...           ...  ...           ...         ...      ...        ...   
95      2             1    1             1  9999-99-99       97          2   
96      2             1    1             1  9999-99-99       97          2   
97      2             1    2             1  9999-99-99       97          2   
98      2             1    2             1  9999-99-99       97          2   
99      2             1    1             1  9999-99-99       97          2   

    AGE  PREGNANT  DIABETES  COPD  ASTHMA  INMSUPR  HIPERTENSION  \
0    65         2         2     2       2        2             1   
1    72        97         2     2       2        2             1   
2    55        97         1     2       2        2             2   
3    53         2         2     2       2        2             2   
4    68        97         1     2       2        2             1   
..  ...       ...       ...   ...     ...      ...           ...   
95   49         2         2     2       2        2             2   
96   24         2         2     2       2        2             2   
97   28        97         2     2       2        2             2   
98   27        97         2     2       2        2             2   
99   26         2         2     2       2        2             2   

    OTHER_DISEASE  CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO  \
0               2               2        2              2        2   
1               2               2        1              1        2   
2               2               2        2              2        2   
3               2               2        2              2        2   
4               2               2        2              2        2   
..            ...             ...      ...            ...      ...   
95              2               2        2              2        2   
96              2               2        2              2        2   
97              2               2        2              2        2   
98              2               2        2              2        1   
99              2               2        2              2        1   

    CLASIFFICATION_FINAL  ICU  
0                      3   97  
1                      5   97  
2                      3    2  
3                      7   97  
4                      3   97  
..                   ...  ...  
95                     7   97  
96                     7   97  
97                     7   97  
98                     7   97  
99                     7   97  

[100 rows x 21 columns]

In [3]:
covid_data.shape

(1048575, 21)

Before Dataset is cleaned, there are 1,048,575 Rows and 21 Columns

In [4]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

### Metadata
>sex: 1 for female and 2 for male.  
>age: of the patient.  
>classification: covid test findings. Values 1-3 mean that the patient was diagnosed with covid in different degrees. 4 or higher means that the patient is not a carrier of covid or that the test is inconclusive.  
>patient type: type of care the patient received in the unit. 1 for returned home and 2 for hospitalization.  
>pneumonia: whether the patient already have air sacs inflammation or not.  
>pregnancy: whether the patient is pregnant or not.  
>diabetes: whether the patient has diabetes or not.  
>copd: Indicates whether the patient has Chronic obstructive pulmonary disease or not.  
>asthma: whether the patient has asthma or not.  
>inmsupr: whether the patient is immunosuppressed or not.  
>hypertension: whether the patient has hypertension or not.  
>cardiovascular: whether the patient has heart or blood vessels related disease.  
>renal chronic: whether the patient has chronic renal disease or not.  
>other disease: whether the patient has other disease or not.  
>obesity: whether the patient is obese or not.  
>tobacco: whether the patient is a tobacco user.  
>usmr: Indicates whether the patient treated medical units of the first, second or third level.  
>medical unit: type of institution of the National Health System that provided the care.  
>intubed: whether the patient was connected to the ventilator.  
>icu: Indicates whether the patient had been admitted to an Intensive Care Unit.  
>date died: If the patient died indicate the date of death, and 9999-99-99 otherwise.  

In [5]:
#remove duplicates 
covid_data_clean = covid_data.drop_duplicates()
covid_data_clean.info()
covid_data_clean.describe() 

<class 'pandas.core.frame.DataFrame'>
Index: 236526 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   USMER                 236526 non-null  int64 
 1   MEDICAL_UNIT          236526 non-null  int64 
 2   SEX                   236526 non-null  int64 
 3   PATIENT_TYPE          236526 non-null  int64 
 4   DATE_DIED             236526 non-null  object
 5   INTUBED               236526 non-null  int64 
 6   PNEUMONIA             236526 non-null  int64 
 7   AGE                   236526 non-null  int64 
 8   PREGNANT              236526 non-null  int64 
 9   DIABETES              236526 non-null  int64 
 10  COPD                  236526 non-null  int64 
 11  ASTHMA                236526 non-null  int64 
 12  INMSUPR               236526 non-null  int64 
 13  HIPERTENSION          236526 non-null  int64 
 14  OTHER_DISEASE         236526 non-null  int64 
 15  CARDIOVASCULAR       

USMER   MEDICAL_UNIT            SEX   PATIENT_TYPE  \
count  236526.000000  236526.000000  236526.000000  236526.000000   
mean        1.512337       7.634230       1.547521       1.577712   
std         0.499849       3.661318       0.497738       0.493925   
min         1.000000       1.000000       1.000000       1.000000   
25%         1.000000       4.000000       1.000000       1.000000   
50%         2.000000       6.000000       2.000000       2.000000   
75%         2.000000      12.000000       2.000000       2.000000   
max         2.000000      13.000000       2.000000       2.000000   

             INTUBED      PNEUMONIA            AGE       PREGNANT  \
count  236526.000000  236526.000000  236526.000000  236526.000000   
mean       44.238401       4.525765      52.620274      54.794826   
std        47.394072      16.731469      19.318869      47.225449   
min         1.000000       1.000000       0.000000       1.000000   
25%         2.000000       1.000000      40.000000       2.000000   
50%         2.000000       2.000000      54.000000      97.000000   
75%        97.000000       2.000000      67.000000      97.000000   
max        99.000000      99.000000     121.000000      98.000000   

            DIABETES           COPD         ASTHMA        INMSUPR  \
count  236526.000000  236526.000000  236526.000000  236526.000000   
mean        2.840855       2.966976       2.949325       3.135630   
std        10.500621       9.882551       9.839247      10.619391   
min         1.000000       1.000000       1.000000       1.000000   
25%         1.000000       2.000000       2.000000       2.000000   
50%         2.000000       2.000000       2.000000       2.000000   
75%         2.000000       2.000000       2.000000       2.000000   
max        98.000000      98.000000      98.000000      98.000000   

        HIPERTENSION  OTHER_DISEASE  CARDIOVASCULAR        OBESITY  \
count  236526.000000  236526.000000   236526.000000  236526.000000   
mean        2.691319       3.687645        2.980767       2.768863   
std        10.098528      12.930442       10.026841       9.959472   
min         1.000000       1.000000        1.000000       1.000000   
25%         1.000000       2.000000        2.000000       1.000000   
50%         2.000000       2.000000        2.000000       2.000000   
75%         2.000000       2.000000        2.000000       2.000000   
max        98.000000      98.000000       98.000000      98.000000   

       RENAL_CHRONIC        TOBACCO  CLASIFFICATION_FINAL            ICU  
count  236526.000000  236526.000000         236526.000000  236526.000000  
mean        2.954956       2.973352              4.781263      44.371008  
std         9.888310      10.289529              1.897253      47.337447  
min         1.000000       1.000000              1.000000       1.000000  
25%         2.000000       2.000000              3.000000       2.000000  
50%         2.000000       2.000000              5.000000       2.000000  
75%         2.000000       2.000000              7.000000      97.000000  
max        98.000000      98.000000              7.000000      99.000000

In [6]:
#dropping missing rows from icu
covid_data_clean = covid_data_clean.drop(covid_data_clean[(covid_data_clean['ICU'] == 97) | (covid_data_clean['ICU'] == 99)].index)
covid_data_clean.head(50)

USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
2        2             1    2             2  09/06/2020        1          2   
5        2             1    1             2  9999-99-99        2          1   
8        2             1    1             2  9999-99-99        2          2   
9        2             1    1             2  9999-99-99        2          2   
11       2             1    2             2  9999-99-99        2          2   
12       2             1    2             2  9999-99-99        2          2   
16       2             1    1             2  9999-99-99        2          1   
20       2             1    2             2  9999-99-99        2          1   
30       2             1    2             2  9999-99-99        2          1   
41       2             1    2             2  9999-99-99        1          1   
53       2             1    2             2  9999-99-99        2          2   
55       2             1    2             2  9999-99-99        2          2   
56       2             1    1             2  9999-99-99        1          1   
57       2             1    1             2  9999-99-99        1          1   
58       2             1    2             2  9999-99-99        2          1   
60       2             1    1             2  9999-99-99        2          2   
61       2             1    1             2  9999-99-99        2          2   
64       2             1    1             2  9999-99-99        2          2   
65       2             1    2             2  9999-99-99        2          2   
66       2             1    2             2  9999-99-99        1          1   
84       2             1    2             2  9999-99-99        2          2   
102      2             1    2             2  9999-99-99        1          1   
103      2             1    2             2  9999-99-99        2          1   
152      1             2    2             2  05/04/2020        1          1   
153      1             2    1             2  08/05/2020        1          2   
154      1             2    2             2  20/05/2020        1          1   
161      1             2    2             2  9999-99-99        2          2   
162      1             2    1             2  9999-99-99        2          1   
163      1             2    2             2  9999-99-99        2          2   
164      1             2    1             2  9999-99-99        2          2   
166      1             2    2             2  9999-99-99        2          1   
188      1             2    1             2  9999-99-99        2          2   
190      1             2    2             2  9999-99-99        2          2   
195      1             2    2             2  9999-99-99        2          1   
196      1             2    2             2  9999-99-99        2          1   
197      1             2    2             2  9999-99-99        2          1   
200      1             2    1             2  9999-99-99        2          2   
204      1             2    1             2  9999-99-99        2          1   
206      1             2    1             2  9999-99-99        2          2   
207      1             2    2             2  9999-99-99        2          2   
208      1             2    2             2  9999-99-99        2          2   
209      1             2    2             2  9999-99-99        2          2   
210      1             2    2             2  9999-99-99        1          1   
297      1             2    1             2  9999-99-99        2          1   
298      1             2    2             2  9999-99-99        2          1   
299      1             2    1             2  9999-99-99        2          2   
300      1             2    2             2  9999-99-99        2          1   
304      1             2    1             2  9999-99-99        2          1   
305      1             2    2             2  9999-99-99        2          2   
307      1             2    2             2  9999-99-99

In [7]:
covid_data_clean.info()
covid_data_clean.describe() 

<class 'pandas.core.frame.DataFrame'>
Index: 130990 entries, 2 to 1048571
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   USMER                 130990 non-null  int64 
 1   MEDICAL_UNIT          130990 non-null  int64 
 2   SEX                   130990 non-null  int64 
 3   PATIENT_TYPE          130990 non-null  int64 
 4   DATE_DIED             130990 non-null  object
 5   INTUBED               130990 non-null  int64 
 6   PNEUMONIA             130990 non-null  int64 
 7   AGE                   130990 non-null  int64 
 8   PREGNANT              130990 non-null  int64 
 9   DIABETES              130990 non-null  int64 
 10  COPD                  130990 non-null  int64 
 11  ASTHMA                130990 non-null  int64 
 12  INMSUPR               130990 non-null  int64 
 13  HIPERTENSION          130990 non-null  int64 
 14  OTHER_DISEASE         130990 non-null  int64 
 15  CARDIOVASCULAR       

USMER   MEDICAL_UNIT            SEX  PATIENT_TYPE  \
count  130990.000000  130990.000000  130990.000000      130990.0   
mean        1.465417       7.247088       1.591885           2.0   
std         0.498805       3.619693       0.491487           0.0   
min         1.000000       1.000000       1.000000           2.0   
25%         1.000000       4.000000       1.000000           2.0   
50%         1.000000       6.000000       2.000000           2.0   
75%         2.000000      12.000000       2.000000           2.0   
max         2.000000      13.000000       2.000000           2.0   

             INTUBED      PNEUMONIA            AGE       PREGNANT  \
count  130990.000000  130990.000000  130990.000000  130990.000000   
mean        1.755325       1.352271      56.754455      58.444148   
std         0.574415       0.477680      18.487942      46.661579   
min         1.000000       1.000000       0.000000       1.000000   
25%         2.000000       1.000000      46.000000       2.000000   
50%         2.000000       1.000000      59.000000      97.000000   
75%         2.000000       2.000000      70.000000      97.000000   
max        99.000000       2.000000     118.000000      98.000000   

            DIABETES           COPD         ASTHMA        INMSUPR  \
count  130990.000000  130990.000000  130990.000000  130990.000000   
mean        2.496435       2.697007       2.721773       2.841194   
std         9.036058       8.504361       8.492472       9.210074   
min         1.000000       1.000000       1.000000       1.000000   
25%         1.000000       2.000000       2.000000       2.000000   
50%         2.000000       2.000000       2.000000       2.000000   
75%         2.000000       2.000000       2.000000       2.000000   
max        98.000000      98.000000      98.000000      98.000000   

        HIPERTENSION  OTHER_DISEASE  CARDIOVASCULAR        OBESITY  \
count  130990.000000  130990.000000   130990.000000  130990.000000   
mean        2.386762       3.332499        2.740079       2.551569   
std         8.723451      11.558455        8.794184       8.725514   
min         1.000000       1.000000        1.000000       1.000000   
25%         1.000000       2.000000        2.000000       2.000000   
50%         2.000000       2.000000        2.000000       2.000000   
75%         2.000000       2.000000        2.000000       2.000000   
max        98.000000      98.000000       98.000000      98.000000   

       RENAL_CHRONIC        TOBACCO  CLASIFFICATION_FINAL            ICU  
count  130990.000000  130990.000000         130990.000000  130990.000000  
mean        2.681716       2.690098              4.488198       1.882564  
std         8.539863       8.748426              1.878366       0.321941  
min         1.000000       1.000000              1.000000       1.000000  
25%         2.000000       2.000000              3.000000       2.000000  
50%         2.000000       2.000000              3.000000       2.000000  
75%         2.000000       2.000000              7.000000       2.000000  
max        98.000000      98.000000              7.000000       2.000000

After Dataset is cleaned, there are 130,990 Rows and 21 Columns

In [8]:
#Analysis of numerical variable
covid_num = covid_data_clean[["AGE"]].copy()
covid_num.describe()

AGE
count  130990.000000
mean       56.754455
std        18.487942
min         0.000000
25%        46.000000
50%        59.000000
75%        70.000000
max       118.000000